In [1]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', as_frame=False)

c:\Users\USER\anaconda3\Lib\site-packages\sklearn\datasets\_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from scipy.ndimage import shift
from sklearn.metrics import accuracy_score


In [3]:
X, y = mnist.data, mnist.target # x-images, y-digits
y = y.astype(np.int8) # labels to numbers

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float64))
X_test_scaled = scaler.transform(X_test.astype(np.float64))

In [6]:
def shift_image(image, direction):
    image = image.reshape(28, 28)
    if direction == "up":
        return shift(image, [1,0], cval=0).reshape(-1)
    if direction == "down":
        return shift(image, [-1,0], cval=0).reshape(-1)
    elif direction == "left":
        return shift(image, [0,1], cval=0).reshape(-1)
    elif direction == "right":
        return shift(image, [0,-1], cval=0).reshape(-1)

In [7]:
def augment_training_set(X_train, y_train):
    X_augmented = []
    y_augmented = []
    
    for i in range(len(X_train)):
        original_image = X_train[i]
        original_label = y_train[i]
        
        X_augmented.append(original_image)
        y_augmented.append(original_label)
        
        for direction in ["up", "down", "left", "right"]:
            shifted_image = shift_image(original_image, direction)
            X_augmented.append(shifted_image)
            y_augmented.append(original_label)
            
    return np.array(X_augmented), np.array(y_augmented)

In [8]:
X_train_augmented, y_train_augmented = augment_training_set(X_train_scaled, y_train)

In [ ]:
param_grid = {
    'n_neighbors': [3, 4, 5, 6, 7],  
    'weights': ['uniform', 'distance'] 
}

In [ ]:
knn_clf = KNeighborsClassifier()
grid_search = GridSearchCV(knn_clf, param_grid, cv=3, verbose=1, n_jobs=-1)

In [ ]:
grid_search.fit(X_train_augmented, y_train_augmented)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


In [ ]:
y_pred = grid_search.best_estimator_.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)

In [ ]:
print("Best hyperparameters:", grid_search.best_params_)
print("Test set accuracy with augmented data", accuracy)

Fitting 3 folds for each of 32 candidates, totalling 96 fits


Best hyperparameters: {'n_neighbors': 4, 'weights': 'distance'}
Test set accuracy: 0.9486428571428571
